In [1]:
import numpy as np
from pyscf import gto, scf

In [2]:
def print_mo_orca_style(mol, mf, nmo=5, nao_print=None):
    """
    Друкує молекулярні орбіталі в стилі ORCA
    
    Параметри:
    -----------
    mol : gto.Mole object
    mf : SCF object
    nmo : int, кількість МО для друку
    nao_print : int, кількість АТ для друку (None = всі)
    """
    mo_energy = mf.mo_energy
    mo_coeff = mf.mo_coeff
    mo_occ = mf.mo_occ
    
    nmo = min(nmo, len(mo_energy))
    if nao_print is None:
        nao_print = mo_coeff.shape[0]
    
    print("\n" + "="*80)
    print("MOLECULAR ORBITALS".center(80))
    print("="*80)
    
    # Друкуємо блоками по 5 орбіталей
    for start_mo in range(0, nmo, 5):
        end_mo = min(start_mo + 5, nmo)
        n_cols = end_mo - start_mo
        
        # Title: MO numbers
        header = "                 "
        for i in range(start_mo, end_mo):
            header += f"{i:>12}"
        print("\n" + header)
        
        # Енергії орбіталей
        energy_line = "                 "
        for i in range(start_mo, end_mo):
            energy_line += f"{mo_energy[i]:>12.5f}"
        print(energy_line)
        
        # Заселеності
        occ_line = "                 "
        for i in range(start_mo, end_mo):
            occ_line += f"{mo_occ[i]:>12.5f}"
        print(occ_line)
        
        # Роздільник
        sep_line = "                 "
        for i in range(n_cols):
            sep_line += "  ----------"
        print(sep_line)
        
        # Коефіцієнти АО
        ao_labels = mol.ao_labels()
        for iao in range(min(nao_print, mo_coeff.shape[0])):
            # Форматуємо мітку АО
            label = ao_labels[iao]
            # Прибираємо зайві пробіли та форматуємо
            label_parts = label.split()
            if len(label_parts) >= 3:
                atom_label = f"{label_parts[1]}{label_parts[0]}"
                ao_label = label_parts[2]
                formatted_label = f"{atom_label:>4} {ao_label:<10}"
            else:
                formatted_label = f"{label:<15}"
            
            line = formatted_label + "  "
            for imo in range(start_mo, end_mo):
                line += f"{mo_coeff[iao, imo]:>12.6f}"
            print(line)
    
    print("\n" + "="*80 + "\n")

In [3]:
mol = gto.M(
    atom = '''
    Li 0 0 0
    ''',
    basis = 'sto-3g',
    spin=1
)

mol.build()
mf = scf.RHF(mol)
mf.kernel()

print_mo_orca_style(mol, mf, nmo=5)

converged SCF energy = -7.31552598128109

                               MOLECULAR ORBITALS                               

                            0           1           2           3           4
                     -2.35349    -0.03896     0.16052     0.16052     0.16052
                      2.00000     1.00000     0.00000     0.00000     0.00000
                   ----------  ----------  ----------  ----------  ----------
 Li0 1s              0.991218   -0.281468    0.000000    0.000000    0.000000
 Li0 2s              0.034144    1.029840    0.000000    0.000000    0.000000
 Li0 2px            -0.000000   -0.000000    1.000000    0.000000    0.000000
 Li0 2py             0.000000   -0.000000    0.000000    1.000000    0.000000
 Li0 2pz             0.000000   -0.000000    0.000000    0.000000    1.000000




In [ ]:
def detailed_mo_analysis(mol, mf, nmo=10):
    """Детальний аналіз МО з додатковою інформацією"""
    
    print_mo_orca_style(mol, mf, nmo)
    
    # Додаткова інформація
    print("\nORBITAL ENERGIES")
    print("-" * 50)
    print(f"{'MO':<6} {'Energy (Eh)':<15} {'Energy (eV)':<15} {'Occ':<6}")
    print("-" * 50)
    
    for i in range(min(nmo, len(mf.mo_energy))):
        energy_ev = mf.mo_energy[i] * 27.2114  # Hartree to eV
        print(f"{i:<6} {mf.mo_energy[i]:<15.6f} {energy_ev:<15.4f} {mf.mo_occ[i]:<6.2f}")
    
    # HOMO-LUMO gap
    homo_idx = np.where(mf.mo_occ > 0)[0][-1]
    lumo_idx = homo_idx + 1
    
    if lumo_idx < len(mf.mo_energy):
        gap_eh = mf.mo_energy[lumo_idx] - mf.mo_energy[homo_idx]
        gap_ev = gap_eh * 27.2114
        print(f"\nHOMO (MO {homo_idx}): {mf.mo_energy[homo_idx]:.6f} Eh")
        print(f"LUMO (MO {lumo_idx}): {mf.mo_energy[lumo_idx]:.6f} Eh")
        print(f"HOMO-LUMO Gap: {gap_eh:.6f} Eh ({gap_ev:.4f} eV)")

In [ ]:
detailed_mo_analysis(mol, mf, nmo=8)


                               MOLECULAR ORBITALS                               

                            0           1           2           3           4
                     -2.35349    -0.03896     0.16052     0.16052     0.16052
                      2.00000     1.00000     0.00000     0.00000     0.00000
                   ----------  ----------  ----------  ----------  ----------
 Li0 1s              0.991218   -0.281468    0.000000    0.000000    0.000000
 Li0 2s              0.034144    1.029840    0.000000    0.000000    0.000000
 Li0 2px            -0.000000   -0.000000    1.000000    0.000000    0.000000
 Li0 2py             0.000000   -0.000000    0.000000    1.000000    0.000000
 Li0 2pz             0.000000   -0.000000    0.000000    0.000000    1.000000



ORBITAL ENERGIES
--------------------------------------------------
MO     Energy (Eh)     Energy (eV)     Occ   
--------------------------------------------------
0      -2.353491       -64.0418        2.00  
1